# Prefeitos e Vereadores Eleits entre 2004-2020

Dados normalizados, agrupados e limpados pelo código abaixo:
https://github.com/turicas/eleicoes-brasil

In [1]:
import pandas as pd

In [112]:
ANO = 2020
CARGO = 'VEREADOR'

In [113]:
cols = ['ano','data_eleicao','totalizacao_turno','turno','codigo_municipio','nome_municipio','sigla_unidade_federativa',
        'numero_partido','sigla_partido','composicao_legenda','cargo',
        'nome_urna','numero_urna','numero_zona','total_votos']

df_chunk = pd.read_csv('input/tse/votacao-zona.csv/votacao-zona.csv', usecols=cols, chunksize=1000000)

chunk_list = []

for chunk in df_chunk:  
    chunk_filter = chunk[chunk.ano == ANO]
    chunk_filter = chunk_filter[chunk_filter.totalizacao_turno.isin(['ELEITO', 'ELEITO POR QP', 'ELEITO POR MEDIA'])]
    chunk_list.append(chunk_filter)  
df_concat = pd.concat(chunk_list)

# ordenar por data da eleicao, pois alguns municipios possuem eleicoes suplementares 
df_concat.sort_values('data_eleicao', ascending=False, inplace=True)

In [114]:
# workaround para deletar zonas duplicadas 
df = df_concat[(df_concat.cargo == CARGO)] \
.groupby(['ano','codigo_municipio','numero_urna','numero_zona'])\
.agg({'nome_municipio':'first','sigla_unidade_federativa':'first',
      'composicao_legenda':'first','numero_partido':'first','sigla_partido':'first',
      'nome_urna':'first','total_votos':'first'})
df.reset_index(inplace=True)

# soma total de votos do candidato em todas as zonas eleitorais do município
df = df \
.groupby(['ano','sigla_unidade_federativa','codigo_municipio','numero_urna']) \
.agg({'nome_municipio':'first','numero_partido':'first','sigla_partido':'first','composicao_legenda':'first',
      'nome_urna':'first','total_votos':'sum'})
df.reset_index(inplace=True)

# alguns municipios tiveram eleicoes suplementares para prefeito, assim manter somente a primeira eleicao
if CARGO == 'PREFEITO':
    df.drop_duplicates('codigo_municipio', keep='first', inplace=True)

In [115]:
df.shape

(55995, 10)

In [116]:
df.sort_values('total_votos', ascending=False).head()

,ano,sigla_unidade_federativa,codigo_municipio,numero_urna,nome_municipio,numero_partido,sigla_partido,composicao_legenda,nome_urna,total_votos
53902,2020,SP,71072,13131,SAO PAULO,13,PT,PT,EDUARDO SUPLICY,167552
53923,2020,SP,71072,25250,SAO PAULO,25,DEM,DEM,MILTON LEITE,132716
53909,2020,SP,71072,15000,SAO PAULO,15,MDB,MDB,DELEGADO PALUMBO,118395
53947,2020,SP,71072,55070,SAO PAULO,55,PSD,PSD,FELIPE BECARI,98717
37469,2020,RJ,60011,50123,RIO DE JANEIRO,50,PSOL,PSOL,TARCISIO MOTTA,86243


In [117]:
if CARGO == 'PREFEITO':
    df.to_csv(f'input/tse/prefeitos_eleitos/{ANO}.csv')
else:
    df.to_csv(f'input/tse/vereadores_eleitos/{ANO}.csv') 